<a href="https://colab.research.google.com/github/santoshk006/Coursera_Capstone/blob/master/Capstone_Project_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IBM Applied Data Science Capstone
##Week 5 Final Report
###Opening a New Shopping Mall in Toronto, Canada

 

*   Scraping Toronto data from Wiki page


*   Cleaning Data
*   GeoCoding
*   Build Toronto map
*   Explore Neghbourhoods using FourSquare API
*   Analyze Neighbourhoods
*   Fetch Shopping Malls data
*   Use K-means clustering (3 clusters)
*   Visualize CLusters
*   Analyze the clusters
*   Observations and Conclusion

#Importing Libraries


In [2]:
import numpy as np

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json

from geopy.geocoders import Nominatim
import geopy.geocoders

import requests
from bs4 import BeautifulSoup

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print("Libraries imported.")

Libraries imported.


#Scraping Data from Wiki page

In [5]:
List_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(List_url).text

In [7]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [9]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#Cleaning Data

In [10]:
df=df[df['Borough']!='Not assigned']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [12]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [13]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [14]:
df_merge.drop_duplicates(inplace=True)

In [15]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [16]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods"
2,M4A,North York,"Victoria Village, Victoria Village"
4,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
6,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
8,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."


In [17]:
df_merge.shape

(103, 3)

#GeoCoding

In [18]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [19]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [20]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [22]:
geo_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge, Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,43.770992,-79.216917,Scarborough,"Woburn, Woburn"
4,M1H,43.773136,-79.239476,Scarborough,"Cedarbrae, Cedarbrae"


In [23]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476


In [30]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West, India Bazaar, ...",43.668999,-79.315572
43,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923
44,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790


In [39]:
toronto_data = toronto_data[['Neighborhood','Latitude', 'Longitude']]
toronto_data.reset_index(inplace = True)
toronto_data = toronto_data[['Neighborhood','Latitude', 'Longitude']]
toronto_data

,Neighborhood,Latitude,Longitude
0,"The Beaches, The Beaches",43.676357,-79.293031
1,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188
2,"India Bazaar, The Beaches West, India Bazaar, ...",43.668999,-79.315572
3,"Studio District, Studio District",43.659526,-79.340923
4,"Lawrence Park, Lawrence Park",43.728020,-79.388790
5,"Davisville North, Davisville North",43.712751,-79.390197
6,"North Toronto West, Lawrence Park, North Toro...",43.715383,-79.405678
7,"Davisville, Davisville",43.704324,-79.388790
8,"Moore Park, Summerhill East, Moore Park, Summe...",43.689574,-79.383160
9,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [40]:
toronto_data.to_csv("toronto_data.csv", index = False)

# Toronto Map

In [42]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto, Canada 43.6534817, -79.3839347.


In [45]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [46]:
# save the map as HTML file
map_toronto.save('map_toronto.html')

# Exploring Neigbourhoods using FourSqaure API

In [47]:
CLIENT_ID = 'XFKYSSZJSJKVIHKCCJERHSMIUYYE05MBAUEJWKXTFWUQLPG2' # Foursquare ID
CLIENT_SECRET = 'R3ZUHYOP3GQU1QGXEDGK5MMYWOB2PJ0ZS4NXNYUSLX0V41NR' # Foursquare Secret
VERSION = '20180604' #Foursquare Version

In [48]:
#Top 100 venues in 2km radius

radius = 2000
LIMIT = 100
venues = []

for lat, lng, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [49]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3878, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"The Beaches, The Beaches",43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,"The Beaches, The Beaches",43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
2,"The Beaches, The Beaches",43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
3,"The Beaches, The Beaches",43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
4,"The Beaches, The Beaches",43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop


In [50]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Berczy Park, Berczy Park",100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place, Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto, Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport, CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",100,100,100,100,100,100
"Central Bay Street, Central Bay Street",100,100,100,100,100,100
"Christie, Christie",100,100,100,100,100,100
"Church and Wellesley, Church and Wellesley",100,100,100,100,100,100
"Commerce Court, Victoria Hotel, Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Davisville North, Davisville North",100,100,100,100,100,100


In [51]:
#Number of unique categories
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 232 uniques categories.


#Analyze Neighbourhoods

In [52]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(3878, 233)


,Neighborhoods,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Non-Profit,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,"The Beaches, The Beaches",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,"The Beaches, The Beaches",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [54]:
#Group by neighbourhood and find mean of frequency of each category

toronto_grouped = toronto_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 233)


,Neighborhoods,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Non-Profit,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,"Berczy Park, Berczy Park",0.00,0.01,0.00,0.00,0.01,0.020000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.00,0.02,0.01,0.01,0.00,0.020000,0.000000,0.00,0.00,0.01,0.00,0.00,0.000000,0.010000,0.010000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.040000,0.000000,0.00,0.00,0.00,0.00,0.00,0.010000,0.00,0.00,0.00,0.00,0.010000,0.000000,0.090000,0.00,0.000000,0.00,0.00,0.01,0.000000,0.000000,0.00,0.01,0.00,0.000000,0.00,0.010000,0.01,0.02,0.00,0.02,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.030000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.01,0.00,0.00,0.000000,0.00,0.01,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.01,0

In [55]:
len(toronto_grouped[toronto_grouped["Shopping Mall"] > 0])

16

# Filtering on Shopping Malls

In [57]:
toronto_malls = toronto_grouped[["Neighborhoods","Shopping Mall"]]
toronto_malls.head()

,Neighborhoods,Shopping Mall
0,"Berczy Park, Berczy Park",0.01
1,"Brockton, Parkdale Village, Exhibition Place, ...",0.00
2,"Business reply mail Processing Centre, South C...",0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00
4,"Central Bay Street, Central Bay Street",0.01


#K-means clustering neighbourhoods with k = 3

In [58]:
kclusters = 3

toronto_clustering = toronto_malls.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 1, 0, 1, 1, 0, 0], dtype=int32)

In [74]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_malls.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

toronto_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
toronto_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,"Berczy Park, Berczy Park",0.01,1
1,"Brockton, Parkdale Village, Exhibition Place, ...",0.00,0
2,"Business reply mail Processing Centre, South C...",0.00,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0
4,"Central Bay Street, Central Bay Street",0.01,1


In [75]:
#Adding coordinates data to the table
toronto_merged = toronto_merged.join(toronto_data.set_index("Neighborhood"), on="Neighborhood")

print(toronto_merged.shape)
toronto_merged.head()

(39, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Berczy Park, Berczy Park",0.01,1,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place, ...",0.00,0,43.636847,-79.428191
2,"Business reply mail Processing Centre, South C...",0.00,0,43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0,43.628947,-79.394420
4,"Central Bay Street, Central Bay Street",0.01,1,43.657952,-79.387383


In [76]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
19,"Little Portugal, Trinity, Little Portugal, Tri...",0.00,0,43.647927,-79.419750
29,"St. James Town, Cabbagetown, St. James Town, C...",0.00,0,43.667967,-79.367675
22,"Parkdale, Roncesvalles, Parkdale, Roncesvalles",0.00,0,43.648960,-79.456325
21,"North Toronto West, Lawrence Park, North Toro...",0.00,0,43.715383,-79.405678
20,"Moore Park, Summerhill East, Moore Park, Summe...",0.00,0,43.689574,-79.383160
18,"Lawrence Park, Lawrence Park",0.00,0,43.728020,-79.388790
17,"Kensington Market, Chinatown, Grange Park, Ken...",0.00,0,43.653206,-79.400049
16,"India Bazaar, The Beaches West, India Bazaar, ...",0.00,0,43.668999,-79.315572
15,"High Park, The Junction South, High Park, The ...",0.00,0,43.661608,-79.464763
14,"Harbourfront East, Union Station, Toronto Isla...",0.00,0,43.640816,-79.381752


#Visualizing the 3 CLusters

In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [78]:
map_clusters.save('map_clusters.html')

# Analyzing the Clusters

In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
19,"Little Portugal, Trinity, Little Portugal, Tri...",0.0,0,43.647927,-79.419750
29,"St. James Town, Cabbagetown, St. James Town, C...",0.0,0,43.667967,-79.367675
22,"Parkdale, Roncesvalles, Parkdale, Roncesvalles",0.0,0,43.648960,-79.456325
21,"North Toronto West, Lawrence Park, North Toro...",0.0,0,43.715383,-79.405678
20,"Moore Park, Summerhill East, Moore Park, Summe...",0.0,0,43.689574,-79.383160
18,"Lawrence Park, Lawrence Park",0.0,0,43.728020,-79.388790
17,"Kensington Market, Chinatown, Grange Park, Ken...",0.0,0,43.653206,-79.400049
16,"India Bazaar, The Beaches West, India Bazaar, ...",0.0,0,43.668999,-79.315572
15,"High Park, The Junction South, High Park, The ...",0.0,0,43.661608,-79.464763
14,"Harbourfront East, Union Station, Toronto Isla...",0.0,0,43.640816,-79.381752


In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
34,"The Annex, North Midtown, Yorkville, The Annex...",0.01,1,43.672710,-79.405678
31,"Stn A PO Boxes, Stn A PO Boxes",0.01,1,43.646435,-79.374846
30,"St. James Town, St. James Town",0.01,1,43.651494,-79.375418
33,"Summerhill West, Rathnelly, South Hill, Forest...",0.01,1,43.686412,-79.400049
0,"Berczy Park, Berczy Park",0.01,1,43.644771,-79.373306
25,"Richmond, Adelaide, King, Richmond, Adelaide, ...",0.01,1,43.650571,-79.384568
24,"Regent Park, Harbourfront, Regent Park, Harbou...",0.01,1,43.654260,-79.360636
37,"Toronto Dominion Centre, Design Exchange, Toro...",0.01,1,43.647177,-79.381576
13,"Garden District, Ryerson, Garden District, Rye...",0.01,1,43.657162,-79.378937
11,"First Canadian Place, Underground city, First ...",0.01,1,43.648429,-79.382280


In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
23,"Queen's Park, Ontario Provincial Government, Q...",0.02,2,43.662301,-79.389494
38,"University of Toronto, Harbord, University of ...",0.02,2,43.662696,-79.400049


# Observations
Most of the shopping malls are present in the central bay area of Toronto, that is in Cluster 0. There are a moderate number in a scattered manner in the outer parts of the city, that is cluster 1. Cluster 2 on the other hand has very less competition, there are only 2 shopping malls in this region - near the University of Toronto. This is a great oppurtunity for realtors to open a new shopping mall in this region, first due to lesser competition and good procimity to the City center.


# Conclusion
Cluster 0 - seem to be suffering from over-competition

Cluster 1 - Donot have much competition, but are scattered and not in the heart of the city

**Cluster 2 - Good area, less competition. Suggested neighbourhood for a new shopping mall - Near University of Toronto**